<a href="https://colab.research.google.com/github/alextanhongpin/python-machine-learning/blob/master/08_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# https://ai.stanford.edu/~amaas/data/sentiment/
!curl https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz -o aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  22.1M      0  0:00:03  0:00:03 --:--:-- 22.1M


In [4]:
!tar -zxf aclImdb_v1.tar.gz

In [5]:
!pip install pyprind

In [6]:
import os

import pandas as pd
import pyprind

In [7]:
basepath = "aclImdb"

labels = {"pos": 1, "neg": 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()

for s in ("test", "train"):
    for l in ("pos", "neg"):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), "r", encoding="utf-8") as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()
df.columns = ["review", "sentiment"]

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:22


In [8]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv("movie_data.csv", index=False, encoding="utf-8")

In [9]:
df = pd.read_csv("movie_data.csv", encoding="utf-8")
df.head(3)

,review,sentiment
0,"This is a well directed Columbo episode, with ...",1
1,This is a movie with an excellent concept for ...,0
2,This is possibly the worst version of the play...,0


In [10]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array(
    [
        "The sun is shining",
        "The weather is sweet",
        "The sun is shining and the weather is sweet",
    ]
)

bag = count.fit_transform(docs)
bag.toarray()

array([[0, 1, 1, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 1, 1],
       [1, 2, 1, 1, 1, 2, 1]])

In [11]:
count.vocabulary_

{'and': 0, 'is': 1, 'shining': 2, 'sun': 3, 'sweet': 4, 'the': 5, 'weather': 6}

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm="l2", smooth_idf=True)
np.set_printoptions(precision=2)
tfidf.fit_transform(count.fit_transform(docs)).toarray()

array([[0.  , 0.43, 0.56, 0.56, 0.  , 0.43, 0.  ],
       [0.  , 0.43, 0.  , 0.  , 0.56, 0.43, 0.56],
       [0.4 , 0.48, 0.31, 0.31, 0.31, 0.48, 0.31]])

In [13]:
df.loc[0, "review"][-50:]  # Print last 50 characters from the review.

' with a better thought out script.<br /><br />7/10'

In [14]:
import re


def preprocessor(text):
    text = re.sub("<[^>]*>", "", text)
    emoticons = re.findall("(?::|;|=)(?:-)?(?:\)|\(|D|P)", text)
    text = re.sub("[\W]+", " ", text.lower()) + " ".join(emoticons).replace("-", "")
    return text

In [15]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [16]:
df.review = df.review.apply(preprocessor)

In [17]:
def tokenizer(text):
    return text.split()


tokenizer("runners like running and thus they run")

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [18]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]


tokenizer_porter("runners like running and thus they run")

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [19]:
import nltk

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
from nltk.corpus import stopwords

stop = stopwords.words("english")
[
    w
    for w in tokenizer_porter("a runner likes runnign and runs a lot")[-10:]
    if w not in stop
]

['runner', 'like', 'runnign', 'run', 'lot']

# Training a logistic regression model for document classification

In [ ]:
X_train = df.loc[:25000, "review"].values
y_train = df.loc[:25000, "sentiment"].values

X_test = df.loc[25000:, "review"].values
y_test = df.loc[25000:, "sentiment"].values

In [24]:
!pip install tune_sklearn

     |████████████████████████████████| 40 kB 3.5 MB/s 
     |████████████████████████████████| 59.6 MB 1.2 MB/s 
     |████████████████████████████████| 173 kB 69.1 MB/s 
     |████████████████████████████████| 124 kB 77.4 MB/s 


In [25]:
# https://towardsdatascience.com/5x-faster-scikit-learn-parameter-tuning-in-5-lines-of-code-be6bdd21833c
from tune_sklearn import TuneSearchCV

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
param_grid = [
    {
        "vect__ngram_range": [(1, 1)],
        "vect__stop_words": [stop, None],
        "vect__tokenizer": [tokenizer, tokenizer_porter],
        "clf__penalty": ["l1", "l2"],
        "clf__C": [1.0, 10.0, 100.0],
    },
    {
        "vect__ngram_range": [(1, 1)],
        "vect__stop_words": [stop, None],
        "vect__tokenizer": [tokenizer, tokenizer_porter],
        "vect__use_idf": [False],
        "vect__norm": [None],
        "clf__penalty": ["l1", "l2"],
        "clf__C": [1.0, 10.0, 100.0],
    },
]

lr_tfidf = Pipeline(
    [("vect", tfidf), ("clf", LogisticRegression(random_state=0, solver="liblinear"))]
)
# gs_lr_tfidf = GridSearchCV(
gs_lr_tfidf = TuneSearchCV(
    lr_tfidf, param_grid, scoring="accuracy", cv=5, verbose=2, n_jobs=-1,
    early_stopping=True,
)
gs_lr_tfidf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/tune_sklearn/tune_basesearch.py:433: UserWarning: early_stopping is enabled but max_iters = 1. To enable partial training, set max_iters > 1.
  category=UserWarning)
/usr/local/lib/python3.7/dist-packages/ray/tune/tune.py:376: UserWarning: The `loggers` argument is deprecated. Please pass the respective `LoggerCallback` classes to the `callbacks` argument instead. See https://docs.ray.io/en/latest/tune/api_docs/logging.html
  "The `loggers` argument is deprecated. Please pass the respective "


Trial _PipelineTrainable_b8f4bed2 reported average_test_score=0.87 with parameters={'early_stopping': True, 'early_stop_type': <EarlyStopping.WARM_START_ITER: 2>, 'X_id': ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000004000000), 'y_id': ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000005000000), 'groups': None, 'cv': StratifiedKFold(n_splits=5, random_state=None, shuffle=False), 'fit_params': {}, 'scoring': {'score': make_scorer(accuracy_score)}, 'max_iters': 1, 'return_train_score': False, 'n_jobs': 1, 'metric_name': 'average_test_score', 'estimator_ids': [ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000006000000), ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000007000000), ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000008000000), ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000009000000), ObjectRef(ffffffffffffffffffffffffffffffffffffffff010000000a000000)], 'vect__ngram_range': (1, 1), 'vect__stop_words': ['i', 'me', 'my', 'my

Trial _PipelineTrainable_b98f0d84 reported average_test_score=0.89 with parameters={'early_stopping': True, 'early_stop_type': <EarlyStopping.WARM_START_ITER: 2>, 'X_id': ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000004000000), 'y_id': ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000005000000), 'groups': None, 'cv': StratifiedKFold(n_splits=5, random_state=None, shuffle=False), 'fit_params': {}, 'scoring': {'score': make_scorer(accuracy_score)}, 'max_iters': 1, 'return_train_score': False, 'n_jobs': 1, 'metric_name': 'average_test_score', 'estimator_ids': [ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000006000000), ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000007000000), ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000008000000), ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000009000000), ObjectRef(ffffffffffffffffffffffffffffffffffffffff010000000a000000)], 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenize

Trial _PipelineTrainable_c704e330 reported average_test_score=0.87 with parameters={'early_stopping': True, 'early_stop_type': <EarlyStopping.WARM_START_ITER: 2>, 'X_id': ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000004000000), 'y_id': ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000005000000), 'groups': None, 'cv': StratifiedKFold(n_splits=5, random_state=None, shuffle=False), 'fit_params': {}, 'scoring': {'score': make_scorer(accuracy_score)}, 'max_iters': 1, 'return_train_score': False, 'n_jobs': 1, 'metric_name': 'average_test_score', 'estimator_ids': [ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000006000000), ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000007000000), ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000008000000), ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000009000000), ObjectRef(ffffffffffffffffffffffffffffffffffffffff010000000a000000)], 'vect__ngram_range': (1, 1), 'vect__stop_words': ['i', 'me', 'my', 'my

In [ ]:
print(f"Best parameter set: {gs_lr_tfidf.best_params_}")
print(f"CV Accuracy: {gs_lr_tfidf.best_score_:.3f}")

clf = gs_lr_tfidf.best_estimator_
print("Test Accuracy: {0:.3f}".format(clf.score(X_test, y_test)))

In [ ]:
import os
import zipfile

if not os.path.isfile("movie_data.csv"):
    if not os.path.isfile("movie_data.csv.zip"):
        print(
            "Please place a copy of the movie_data.csv.gz"
            "in this directory. You can obtain it by"
            "a) executing the code in the beginning of this"
            "notebook or b) by downloading it from GitHub:"
            "https://github.com/rasbt/python-machine-learning-"
            "book-2nd-edition/blob/master/code/ch08/movie_data.csv.gz"
        )
    else:
        with zipfile.ZipFile("movie_data.csv.zip", "r") as zip_ref:
            zip_ref.extractall(".")

In [ ]:
import re

import numpy as np
from nltk.corpus import stopwords

stop = stopwords.words("english")


def tokenizer(text):
    text = re.sub("<[^>]*>", "", text)
    emoticons = re.findall("(?::|;|=)(?:-)?(?:\)|\(|D|P)", text)
    text = re.sub("[\W]+", " ", text.lower()) + " ".join(emoticons).replace("-", "")
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [ ]:
def stream_docs(path):
    with open(path, "r", encoding="utf-8") as csv:
        next(csv)  # Skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [ ]:
next(stream_docs(path="movie_data.csv"))

('"My family and I normally do not watch local movies for the simple reason that they are poorly made, they lack the depth, and just not worth our time.<br /><br />The trailer of ""Nasaan ka man"" caught my attention, my daughter in law\'s and daughter\'s so we took time out to watch it this afternoon. The movie exceeded our expectations. The cinematography was very good, the story beautiful and the acting awesome. Jericho Rosales was really very good, so\'s Claudine Barretto. The fact that I despised Diether Ocampo proves he was effective at his role. I have never been this touched, moved and affected by a local movie before. Imagine a cynic like me dabbing my eyes at the end of the movie? Congratulations to Star Cinema!! Way to go, Jericho and Claudine!!"',
 1)

In [ ]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(
    decode_error="ignore", n_features=2 ** 21, preprocessor=None, tokenizer=tokenizer
)

clf = SGDClassifier(loss="log", random_state=1, max_iter=1)
doc_stream = stream_docs(path="movie_data.csv")

In [ ]:
import pyprind

pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:30


In [ ]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print("Accuracy: {0:.3f}".format(clf.score(X_test, y_test)))

Accuracy: 0.866


In [ ]:
clf = clf.partial_fit(X_test, y_test)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words="english", max_df=0.1, max_features=5000)
X = count.fit_transform(df["review"].values)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(
    n_components=10, random_state=123, learning_method="batch"
)
X_topics = lda.fit_transform(X)

In [ ]:
lda.components_.shape

(10, 5000)

In [ ]:
n_top_words = 5
feature_names = count.get_feature_names()

for topic_idx, topic in enumerate(lda.components_):
    print("Topic {0}".format(topic_idx + 1))
    print(
        " ".join(
            [feature_names[i] for i in topic.argsort()[: -n_top_words - 1 : -1]]
        )  # Sort in reverse orders
    )

Topic 1
worst minutes awful script stupid
Topic 2
family mother father girl children
Topic 3
american dvd war music tv
Topic 4
human audience cinema art feel
Topic 5
police guy car dead murder
Topic 6
horror house gore blood sex
Topic 7
role performance comedy actor performances
Topic 8
series episode war episodes season
Topic 9
book version original effects read
Topic 10
action fight guy guys cool


In [ ]:
horror = X_topics[:, 5].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
    print("Horror movie #{0}".format(iter_idx + 1))
    print(df["review"][movie_idx][:300], "...")

Horror movie #1
Emilio Miraglia's first Giallo feature, The Night Evelyn Came Out of the Grave, was a great combination of Giallo and Gothic horror - and this second film is even better! We've got more of the Giallo side of the equation this time around, although Miraglia doesn't lose the Gothic horror stylings tha ...
Horror movie #2
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...
Horror movie #3
This film marked the end of the "serious" Universal Monsters era (Abbott and Costello meet up with the monsters later in "Abbott and Costello Meet Frankentstein"). It was a somewhat desparate, yet fun attempt to revive the classic monsters of the Wolf Man, Frankenstein's monster, and Dracula one "la ...


# Serializing fitted scikit-learn estimators

- training machine learning is computationally expensive
- pickled allow us to serialize/deserialize Python object structures to compact bytecode

In [ ]:
import os
import pickle

dest = os.path.join("movieclassifier", "pkl_objects")
if not os.path.exists(dest):
    os.makedirs(dest)


pickle.dump(stop, open(os.path.join(dest, "stopwords.pkl"), "wb"), protocol=4)
pickle.dump(clf, open(os.path.join(dest, "classifier.pkl"), "wb"), protocol=4)